# Deep Convolutional Neural Networks Classification

**Importación de bibliotecas para análisis y escalado de datos**

In [1]:
import torch

import torch.nn as nn
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

Se configuran las transformaciones necesarias para convertir las imágenes en tensores.

Después, se carga el dataset FashionMNIST con imágenes de entrenamiento y validación, se divide el dataset original en conjuntos de entrenamiento y validación.

Y por último, se carga el dataset de prueba sin volver a descargar las imágenes.

In [2]:
import torchvision
from torchvision import transforms
image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
])
mnist_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=True,
    transform=transform, download=True
)
from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, 
                             torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, 
                             torch.arange(
                                 10000, len(mnist_dataset)
                            ))
mnist_test_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=False,
    transform=transform, download=False
)

100%|██████████| 26.4M/26.4M [00:04<00:00, 5.88MB/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 819kB/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 4.16MB/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 10.0MB/s]


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



Se configura el tamaño del batch a 64 y se establecen las semillas aleatorias, además de crear 2 loaders, uno para el dataset de entrenamiento y otro para el de validación.

In [3]:
from torch.utils.data import DataLoader
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset,
                      batch_size,
                      shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset,
                      batch_size,
                      shuffle=False)

**Construyendo una CNN en PyTorch**

Se construye el modelo de la CNN y una segunda capa convolucional y de activación.

In [4]:
model = nn.Sequential()
model.add_module(
    'conv1',
    nn.Conv2d(
        in_channels=1, out_channels=32,
        kernel_size=5, padding=2
    )
)
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels=32, out_channels=64,
        kernel_size=5, padding=2
    )
)
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

Se verifican las dimensiones después de haber creado las capas convolucionales y de pooling.

In [5]:
x = torch.ones((4, 1, 28, 28))
model(x).shape

torch.Size([4, 64, 7, 7])

Como se puede observar, las dimensiones de entrada de 28x28 se han reducido a 7x7 debido a la aplicación secuencial de las capas de convolución, activación y pooling.

Una vez hecho esto, se aplana el tensor para aquellas capas que se encuentren completamente conectadas.

In [6]:
model.add_module('flatten', nn.Flatten())
x = torch.ones((4, 1, 28, 28))
model(x).shape

torch.Size([4, 3136])

Como se puede observar, el tensor ha pasado de tener dimensiones 64x7x7 a 3136, por lo que ya está listo para ser procesado por aquellas capas que se encuentren totalmente conectadas al modelo.

Después, se añaden las capas conectadas (fully connected) y de regularización.

In [7]:
model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2', nn.Linear(1024, 10))
model = model.cpu()
# model = model.cuda()

Se configuran tanto la función de pérdida como el optimizador.

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.cpu()
            y_batch = y_batch.cpu()
            # x_batch = x_batch.cuda()
            # y_batch = y_batch.cuda()
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)
            is_correct = (
                torch.argmax(pred, dim=1) == y_batch
            ).float()
            accuracy_hist_train[epoch] += is_correct.sum()
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.cpu()
                y_batch = y_batch.cpu()
                # x_batch = x_batch.cuda()
                # y_batch = y_batch.cuda()
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += \
                    loss.item()*y_batch.size(0)
                is_correct = (
                    torch.argmax(pred, dim=1) == y_batch
                ).float()
                accuracy_hist_valid[epoch] += is_correct.sum()
        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
        
        print(f'Epoch {epoch+1} accuracy: '
              f'{accuracy_hist_train[epoch]:.4f} val_accuracy: '
              f'{accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train, loss_hist_valid, \
        accuracy_hist_train, accuracy_hist_valid

---

**Entrenamiento de la CNN (1)**

In [10]:
# torch.manual_seed(1)
# num_epochs = 20
# hist = train(model, num_epochs, train_dl, valid_dl)

Las métricas impresas durante el entrenamiento muestran cómo evolucionan la pérdida y la precisión tanto en los conjuntos de entrenamiento como de validación.

Es importante identificar si hay problemas como sobreajuste, observando la brecha entre la precisión de entrenamiento y validación.

In [11]:
# x_arr = np.arange(len(hist[0])) + 1
# fig = plt.figure(figsize=(12,4))
# ax = fig.add_subplot(1, 2, 1)
# ax.plot(x_arr, hist[0], '--o', label='Train loss')
# ax.plot(x_arr, hist[1], '--<', label='Validation loss')
# ax.legend(fontsize=15)
# ax = fig.add_subplot(1, 2, 2)
# ax.plot(x_arr, hist[2], '--o', label='Train acc.')
# ax.plot(x_arr, hist[3], '--<', label='Validation acc.')
# ax.legend(fontsize=15)
# ax.set_xlabel('Epoch', size=15)
# ax.set_ylabel('Accuracy', size=15)
# plt.show()

In [12]:
# pred = model(mnist_test_dataset.data.unsqueeze(1) / 255.)
# is_correct = (
#     torch.argmax(pred, dim=1) == mnist_test_dataset.targets
# ).float()
# print(f'Test accuracy: {is_correct.mean():4f}')

Como se puede observar, los gráficos de pérdida y precisión permiten evaluar visualmente el rendimiento del modelo.

Una convergencia estable indica que el modelo ha aprendido patrones relevantes sin sobreajustarse a los datos de entrenamiento.

---

**Entrenamiento de la CNN (2)**

In [13]:
# torch.manual_seed(1)
# num_epochs = 20
# hist = train(model, num_epochs, train_dl, valid_dl)

Las métricas impresas durante el entrenamiento muestran cómo evolucionan la pérdida y la precisión tanto en los conjuntos de entrenamiento como de validación.

Es importante identificar si hay problemas como sobreajuste, observando la brecha entre la precisión de entrenamiento y validación.

In [14]:
# plt.plot(range(1, num_epochs+1), hist[0], label='Train loss')
# plt.plot(range(1, num_epochs+1), hist[1], label='Valid loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Train loss')
# plt.legend()
# plt.show()

In [15]:
# plt.plot(range(1, num_epochs+1), hist[2].cpu().numpy(), label='Accuracy train')
# plt.plot(range(1, num_epochs+1), hist[3].cpu().numpy(), label='Accuracy valid')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Accuracy')
# plt.legend()
# plt.show()

Como se puede observar, los gráficos de pérdida y precisión permiten evaluar visualmente el rendimiento del modelo.

Una convergencia estable indica que el modelo ha aprendido patrones relevantes sin sobreajustarse a los datos de entrenamiento.